# [실습] LangChain으로 OpenAI GPT와 Google Gemini API 사용하기

LangChain(랭체인)은 LLM 기반의 어플리케이션을 효율적으로 개발할 수 있게 해주는 라이브러리입니다.



LangChain은 GPT, Gemini 등의 API와 HuggingFace, Ollama 등의 오픈 모델 환경 모두에서 사용할 수 있습니다.

이번 실습에서는 대표적인 LLM인 Google Gemini와 OpenAI GPT의 API를 사용해 진행하겠습니다.    

Gemini는 무료 사용량이 존재하지만, GPT는 유료 API 크레딧이 필요합니다.   
만약 유료 크레딧이 없으신 분들은 Gemini만으로 진행해 주세요.

In [ ]:
!pip install langchain langchain-community langchain-google-genai langchain-openai

Google Colab 환경이 아닌 경우에는, 아래 라이브러리도 설치해야 할 수 있습니다.

In [ ]:
# !pip install google-generativeai

## LLM

LangChain에서, LLM을 부르는 방법은 주로 `ChatOpenAI`, `ChatGoogleGenerativeAI`와 같은 개별 클래스를 불러오거나,   
`init_chat_model`을 통해 Provider와 모델 이름을 전달하는 방식으로 이루어집니다.

### API 키 준비하기


Google API 키를 등록하고 입력합니다.   
구글 계정 로그인 후 https://aistudio.google.com  에 접속하면, API 키 생성이 가능합니다.   

OpenAI API 키는 유료 계정 로그인 후   
https://platform.openai.com/api-keys 에 접속하면 생성이 가능합니다.    
(유료 계정과 무관하게, 크레딧 결제가 필요합니다.)

In [ ]:
import os

os.environ['GOOGLE_API_KEY']="AIxxx"

os.environ['OPENAI_API_KEY'] = 'sk-...'

Google AI Studio의 `Create Prompt`에서, 모델 목록과 무료 API 사용량을 확인할 수 있습니다.

OpenAI 모델의 목록과 가격은 https://openai.com/api/pricing/ 에서 확인할 수 있습니다.

## LLM

chat 모델 사용을 위해 ChatGoogleGenerativeAI, ChatOpenAI를 불러오겠습니다.

모델마다 다른 Safety 등의 요소를 제외하고, 공통적으로 아래의 파라미터를 갖습니다.
- model : 모델의 이름입니다.
- temperature : 모델 출력의 무작위성을 결정합니다. 0부터 2 사이의 값을 지정할 수 있으며,   
숫자가 클수록 무작위 출력이 증가합니다.    
(o3-mini, o1 등의 Reasoning 모델은 지원하지 않는 경우도 있습니다)

- max_tokens : 출력의 최대 길이를 지정합니다. 해당 토큰 수가 넘어가면 출력이 중간에 종료됩니다.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature = 0.7,
    max_tokens = 2048
)

In [ ]:
from langchain_openai import ChatOpenAI

llm_gpt = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 1.0,
    max_tokens = 2048
)

LangChain은 프롬프트, LLM, 체인 등의 구성 요소를 서로 연결하는 방식으로 구성됩니다.  
각각의 요소를 `Runnable`이라고 부르는데요.   
`Runnable`은  `invoke()`를 통해 실행합니다.

In [ ]:
question = '''프롬프트 엔지니어링의 핵심적인 요소 4개가 뭔가요?'''

response = llm.invoke(question)
response

In [ ]:
print(response.content)

위처럼 문자열을 그대로 입력하게 되면, 해당 문자열은 HumanMessage 클래스로 변환되어 입력됩니다.   
HumanMessage에 대한 출력 형식은 AIMessage 클래스로 정의됩니다.

In [ ]:
question = '''울림을 주는 2000년대 영화 명대사를 하나 알려주세요.
대사가 나온 배경과 의미도 한 문장으로 설명해 주세요.'''
response = llm.invoke(question)

print('# Gemini-2.0-Flash의 답변:', response.content)

만약, 여러 개의 모델을 불러오고 싶은 경우에는 아래와 같이 공통 인터페이스를 사용할 수도 있습니다.

In [ ]:
from langchain.chat_models import init_chat_model

gpt_4o = init_chat_model("gpt-4o", model_provider="openai", temperature = 1.0)
gemini_2_0_flash = init_chat_model("gemini-2.0-flash", model_provider="google_genai", temperature = 1.0)

## 스트리밍

스트리밍은 모델을 토큰이 생성되는 순서대로 출력하는 방법입니다.

In [ ]:
import time
chunks = []
for chunk in llm.stream("5문장으로 당신을 소개해주세요. 매 문장마다 줄을 띄우세요."):
    #time.sleep(0.4)
    print(chunk.content, end="", flush=True)

실제 환경에서는 프롬프트의 형태를 사전에 설정하고,   
같은 형태로 입력 변수가 주어질 때마다 프롬프트를 작성하게 하는 것이 효율적입니다.

## Prompt Template

LangChain은 프롬프트의 템플릿을 구성할 수 있습니다.

In [ ]:
from langchain_core.prompts import PromptTemplate

explain_template = """당신은 주어진 단어에 대해, 유머러스하게 한 문장으로 표현합니다.

제시어: {word}"""
print(explain_template)

In [ ]:
explain_prompt = PromptTemplate(template = explain_template)

explain_prompt.format(word = "트랜스포머 네트워크")

In [ ]:
llm.invoke(explain_prompt.format(word = "트랜스포머 네트워크")).content

두 개의 매개변수를 받아 프롬프트를 만들어 보겠습니다.

In [ ]:
translate_template = "{topic}에 대해 {language}로 설명하세요."

translate_prompt = PromptTemplate(template = translate_template)

translate_prompt.format(topic='torschlusspanik', language='초등학생을 위한 한국어')

In [ ]:
X = translate_prompt.format(topic='torschlusspanik', language='한국어')
response = llm.invoke(X)
print(response.content)

## Chat Prompt Template

Web UI를 통해 ChatGPT, Claude 등의 LLM을 실행하는 경우와 다르게,   
API의 호출은 유저 메시지 이외의 다양한 메시지를 사용할 수 있습니다.   
- system: AI 모델의 행동 방식을 결정하는 시스템 메시지
- user(human): 사용자의 메시지
- ai(assistant): AI 모델의 메시지

이는 LangChain 내부에서 모델에 맞는 템플릿으로 변환되어 입력됩니다.   

Ex) 라마 3 시리즈의 템플릿
```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>
```

Qwen 시리즈의 템플릿
```
<|im_start|>system
You are a helpful AI assistant
<|im_end|>
<|im_start|>user
Hello!
<|im_end|>
<|im_start|>assistant
```

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ("system", '당신은 항상 부정적인 말만 하는 챗봇입니다. 첫 문장은 항상 사용자의 의견을 반박하고, 이후 대안을 제시하세요.'),
    ("user", '{A} 너무 좋은 것 같아요!')
    # system, user = human, ai = assistant
]
)
prompt.format_messages(A='LangChain')

In [ ]:
llm.invoke(prompt.format_messages(A='LangChain'))

또는, 아래와 같이 메시지를 직접 불러와 사용할 수도 있습니다.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

topic=''

msgs = [
    SystemMessage('항상 다섯 단어로 표현하세요.'),
     HumanMessage(f'{topic}에 대해 설명해줘!')
]

response = llm.invoke(msgs)
print('# Gemini-2.0-Flash의 답변:', response.content)

response = llm_gpt.invoke(msgs)
print('# GPT-4o-mini의 답변:', response.content)


## Few-Shot Prompting
모델이 참고할 예시를 포함하는 퓨 샷 프롬프팅은
   
   모델 출력의 형식과 구조를 효과적으로 변화시킬 수 있습니다.  


Few-Shot Prompt Template을 이용해 example을 프롬프트에 추가해 보겠습니다.

In [ ]:
# 예시 : Prompt Example 2개
from langchain_core.prompts.few_shot import FewShotPromptTemplate

examples = [
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
    {
    "question": "Who won more Grammy Awards, Beyoncé or Michael Jackson?",
    "answer": """
Are follow up questions needed here: Yes.
Follow up: How many Grammy Awards has Beyoncé won?
Intermediate answer: Beyoncé has won 32 Grammy Awards.
Follow up: How many Grammy Awards did Michael Jackson win?
Intermediate answer: Michael Jackson won 13 Grammy Awards.
So the final answer is: Beyoncé
""",
    }
]

Example 데이터를 구성할 템플릿을 만듭니다.

In [ ]:
example_prompt = PromptTemplate(template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

위에서 만든 Examples와 템플릿, prefix와 suffix를 이용해 전체 템플릿을 만들 수 있습니다.

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,

    prefix="질문-답변 형식의 예시가 주어집니다. 같은 방식으로 답변하세요.",
    suffix="Question: {input}",
    #prefix, suffix : Optional

)
print(prompt.format(input=""))

In [ ]:
question = "Current Date : 2025. April. What is the age of the director of the movie which won the best international film in Oscar in 2018?"
X = prompt.format(input=question)
print(llm.invoke(X).content)

# LangChain으로 이미지 입력하기

이미지와 같은 멀티모달 입력의 경우, 텍스트와 구분하여 Dict 형식으로 입력됩니다.   
URL을 직접 전달하거나, 파일을 전달하는 경우에 따라 코드가 달라집니다.

In [ ]:
image_url = 'https://images.pexels.com/photos/1851164/pexels-photo-1851164.jpeg'
from IPython.display import Image
import requests

# 이미지 출력
img = Image(url = image_url, width = 400)
img

In [ ]:
# 1. URL에서 전달하기
image_prompt = ChatPromptTemplate([
    ('user',[
                {"type": "text", "text": "{question}"},

                {"type": "image_url",
                    "image_url": {"url": image_url}
                }
             ]
     )])
X = image_prompt.format_messages(question= '이 사진에 대해 묘사해 주세요.')
print(llm.invoke(X).content)

In [ ]:
import base64
import httpx

# 이미지 URL에서 데이터 받아오기
image_url = 'https://cloud.google.com/static/vertex-ai/generative-ai/docs/multimodal/images/timetable.png?hl=ko'
response = httpx.get(image_url)

image_data = base64.b64encode(response.content).decode("utf-8")

with open('picture.jpeg', 'wb') as file:
    file.write(response.content)

In [ ]:
# 2. 로컬 폴더에서 이미지 읽어보기
with open('./picture.jpeg', 'rb') as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')


image_prompt = ChatPromptTemplate([
    ('user',[
                {"type": "text", "text": "{question}"},

                {"type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}
                }
             ]
     )])

X = image_prompt.format_messages(question='이 그림에 대해 한국어로 설명해 주세요.')

print(llm.invoke(X).content)

멀티모달 입력의 경우, 적절한 프롬프트가 더 중요합니다.

In [ ]:
X = image_prompt.format_messages(question="""
이 이미지에 표시된 공항 보드에서
시간과 도시를 분석해서 목록으로 표시해 주세요.
형식은 시간 - 도시입니다.
예시) 12:00 - 런던
13:00 - 서울

목록만 출력하세요.""")

print(llm.invoke(X).content)